In [68]:
# logging utility

import time
from datetime import datetime

LOG_FILE = "latent_experiment_log.txt"

def log_message(message):
    with open(LOG_FILE, "a") as f:
        f.write(message + "\n")


In [69]:
# --------------------------------------------------------------
# Latent-Space Face Impersonation (Reduced-Compute Variant)
# MSc Dissertation - Stella Williams
# Inversion-free latent target synthesis experiment
# Multi-run evaluation for repeatability and target specificity
# --------------------------------------------------------------




In [70]:
import os
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1




In [71]:
"Cell 2: Device, FaceNet setup"

# --------------------------------------------------------------
# Device configuration
# --------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------------------------------------------------------------
# FaceNet embedding model (verification system)
# --------------------------------------------------------------

facenet = InceptionResnetV1(
    pretrained="vggface2"
).eval().to(device)

# --------------------------------------------------------------
# Preprocessing for FaceNet (160x160 input requirement)
# --------------------------------------------------------------

to_facenet = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

def load_image(path):
    return Image.open(path).convert("RGB")

def img_to_tensor(img):
    return to_facenet(img).unsqueeze(0).to(device)

@torch.no_grad()
def facenet_embed(img_tensor):
    emb = facenet(img_tensor)
    return nn.functional.normalize(emb, p=2, dim=1)

def cosine(a, b):
    return (a * b).sum(dim=1)



Using device: cpu


In [72]:
# --------------------------------------------------------------
# Load pretrained StyleGAN2-ADA generator
# --------------------------------------------------------------
import sys
sys.path.append("/Users/stel/Documents/Dissertation/msc-biometric-security-clean/stylegan2-ada-pytorch")



import pickle

STYLEGAN_PKL = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Models/stylegan2-ffhq.pkl"

with open(STYLEGAN_PKL, "rb") as f:
    G = pickle.load(f)["G_ema"].to(device).eval()

print("Loaded StyleGAN2 generator")





Loaded StyleGAN2 generator


In [73]:
# --------------------------------------------------------------
# Generator wrapper
# Converts single w latent into w+ internally
# --------------------------------------------------------------

def synth_from_w_single(w):
    """
    Accepts either:
    - w shape [1, w_dim]
    - w shape [1, num_ws, w_dim] (w+)

    Returns image in [0,1]
    """

    if w.ndim == 2:
        # w is [1, w_dim] → broadcast to w+
        w_plus = w.unsqueeze(1).repeat(1, G.synthesis.num_ws, 1)
    elif w.ndim == 3:
        # Already w+
        w_plus = w
    else:
        raise ValueError(f"Unexpected latent shape: {w.shape}")

    img = G.synthesis(w_plus, noise_mode="const")
    img = (img + 1) / 2
    return img.clamp(0, 1)



In [74]:
# --------------------------------------------------------------
# Load target identity (impersonation objective)
# --------------------------------------------------------------

TARGET_A_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Lenny_Kravitz/Lenny_Kravitz_0001.jpg"
TARGET_B_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Patricia_Clarkson/Patricia_Clarkson_0001.jpg"

targetA_img = load_image(TARGET_A_PATH)
targetB_img = load_image(TARGET_B_PATH)

targetA_t = img_to_tensor(targetA_img)
targetB_t = img_to_tensor(targetB_img)

with torch.no_grad():
    e_target = facenet_embed(targetA_t)
    e_non_target = facenet_embed(targetB_t)

print("Target embedding norm:", e_target.norm().item())

print("Initial w shape:", w.shape)
#inspecting the shape, it is likely [1, 1, w_dim], or [1, num_ws, w_dim] - with the latter meaning
#we are alreadz in w+ space and the wrapper must not broadcast again. in my previous version, no loop,
#i initialised w differently, mapping for from a random latent or mapping. dimensions changed.


Target embedding norm: 1.0
Initial w shape: torch.Size([1, 512])


In [75]:

log_message("--------------------------------------------------")
log_message(f"Experiment started: {datetime.now()}")
log_message("Reduced-compute latent impersonation batch")
log_message("--------------------------------------------------")


In [76]:
# --------------------------------------------------------------
# Load non-target identity (Target B)
# --------------------------------------------------------------
NON_TARGET_PATH = "/Users/stel/Documents/Dissertation/msc-biometric-security-clean/Datasets/lfw/Patricia_Clarkson/Patricia_Clarkson_0001.jpg"

non_target_img = load_image(NON_TARGET_PATH)
non_target_t = img_to_tensor(non_target_img)

with torch.no_grad():
    e_nontarget = facenet_embed(non_target_t)


In [ ]:
# --------------------------------------------------------------
# Multi-Run Latent Impersonation Experiment   
# --------------------------------------------------------------    
NUM_RUNS = 5
MAX_STEPS = 80
TARGET_SIM = 0.85
LR = 0.03

results = []
best_example = None

print("\nRunning latent impersonation batch...\n")

for run in range(NUM_RUNS):

    print(f"\n=== Run {run+1}/{NUM_RUNS} ===")
    log_message(f"\n=== Run {run+1}/{NUM_RUNS} ===")

    torch.manual_seed(run)

    # Initialise latent
    with torch.no_grad():
        w = G.mapping.w_avg.unsqueeze(0).to(device)

    w = w.clone().detach().requires_grad_(True)

    optimizer = optim.Adam([w], lr=0.03)

    start_time = time.time()
    success = False

    for step in tqdm(range(MAX_STEPS), desc=f"Run {run+1}", leave=True):

        img = synth_from_w_single(w)
        img_160 = nn.functional.interpolate(img, (160,160))

        emb = facenet(img_160)
        emb = nn.functional.normalize(emb, dim=1)

        sim_target = cosine(emb, e_target)
        sim_nontarget = cosine(emb, e_nontarget)

        loss = 1 - sim_target.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 10 == 0:
            msg = f"Run {run+1} | Step {step} | cos_target={sim_target.item():.4f} | cos_non_target={sim_nontarget.item():.4f}"
            print(msg)
            log_message(msg)

        if sim_target.item() >= TARGET_SIM:
            success = True
            break

    runtime = time.time() - start_time

    # Final evaluation
    final_target = sim_target.item()
    final_non_target = sim_nontarget.item()

    summary = (
        f"Run {run+1} COMPLETE | "
        f"Success={success} | "
        f"Steps={step} | "
        f"Final_target={final_target:.4f} | "
        f"Final_non_target={final_non_target:.4f} | "
        f"Runtime_sec={runtime:.2f}"
    )

    print(summary)
    log_message(summary)




Running latent impersonation batch...


=== Run 1/5 ===


Run 1:   1%|▍                                | 1/80 [08:58<11:49:09, 538.60s/it]

Run 1 | Step 0 | cos_target=-0.0156 | cos_non_target=0.1060


Run 1:  14%|████▎                          | 11/80 [1:14:22<7:26:05, 387.91s/it]

Run 1 | Step 10 | cos_target=0.7614 | cos_non_target=0.0849


Run 1:  19%|█████▊                         | 15/80 [1:54:55<8:17:59, 459.68s/it]


Run 1 COMPLETE | Success=True | Steps=15 | Final_target=0.8527 | Final_non_target=0.0180 | Runtime_sec=6895.30

=== Run 2/5 ===


Run 2:   1%|▍                                 | 1/80 [06:19<8:19:03, 379.03s/it]

Run 2 | Step 0 | cos_target=-0.0156 | cos_non_target=0.1060


Run 2:   4%|█▎                                | 3/80 [18:47<8:01:34, 375.25s/it]

In [ ]:
# ----------------------------------
# Summary statistics
# ----------------------------------

success_rate = sum(r["success"] for r in results) / NUM_RUNS
mean_steps = np.mean([r["steps"] for r in results])
mean_cos = np.mean([r["final_sim_target"] for r in results])
mean_non_target = np.mean([r["final_sim_non_target"] for r in results])
mean_runtime = np.mean([r["runtime_sec"] for r in results])

print("\n==============================")
print("Batch Summary")
print("==============================")
print(f"Success rate: {success_rate*100:.1f}%")
print(f"Mean steps: {mean_steps:.2f}")
print(f"Mean final cosine (target): {mean_cos:.4f}")
print(f"Mean cosine (non-target): {mean_non_target:.4f}")
print(f"Mean runtime: {mean_runtime/60:.2f} minutes")


In [ ]:
# --------------------------------------------------------------
# Visualise final generated adversarial face
# --------------------------------------------------------------

if best_example is not None:
    plt.figure(figsize=(5,5))
    plt.imshow(best_example.squeeze().permute(1,2,0))
    plt.title("Example successful latent impersonation")
    plt.axis("off")
    plt.show()

